# ECON 490: Locals and Globals (4)

## Prerequisites 

1. View the characteristics of any dataset using the command `describe`.
2. Use `help` to learn how to run commands.
3. Understand the Stata command syntax using the command `summarize`.
4. Create loops using the commands `for`, `while`, `forvalues` and `foreach`.

## Learning Outcomes

1. Recognize the difference between data set variables and Stata variables.
2. Recognize the difference between local and global Stata variables.
3. Use the command `local` to create temporary macros.
4. Use the command `global` to create permanent macros.
5. Forecast how you will use macros in your own research.

## 4.1 Stata Variables

In ECON 325 and ECON 326, you learned that "variables" are characteristics of a data set. For example, if we had a data set that included all of the countries in the world, we might have a variable which indicates each country's population. As another example, if we had a data set that included a sample of persons in Canada, we might have a variable which indicates each person's marital status. These are data set variables, and they can be qualitative (strings) or quantitative (numeric). 

In Stata, there is a separate category of variables available for use which we call "macros". Macros work as placeholders for values that we want to store either temporarily or permanently. Locals are macros that store data temporarily, while globals are macros that store data permanently, or at least as long as we have Stata open on our computer. We can think of Stata macros as analogous to workspace objects in Python or R. Below, you are going to learn how to use these macros in your own research.

## 4.2 Locals 

The first use of local macros is to store results of your code. To help you understand how powerful this is, you should be aware that most Stata commands have hidden results stored after they are run. Consider the following example

In [ ]:
sysuse auto,clear

summarize price


When we ran `summarize` above, Stata produced output that was stored in several local variables. We can access those stored results with the command `return list` (for regular commands) or `ereturn list` (for estimation commands, which we'll cover later in Module 12). Since `summarize` is not an estimation command, we can run the following:

In [ ]:
return list

Notice that Stata has reported that variables have been stored as scalars, where a scalar is simply a quantity. 

If we want Stata to tell us the mean price from the automobile data set that was just calculated using `summarize`, we can use the following:

In [ ]:
display r(mean)

We can now store that scalar as a local, and use that local in other Stata commands:

In [ ]:
local price_mean = r(mean)
display "The mean of price variable is `price_mean'." 

Imagine that we wanted to create a new variable that is equal to the price minus the mean of that same variable. We would do this if we wanted to de-mean that variable or, in other words, create a new price variable that has a mean of zero. To do this, we could use the `generate` command along with the local we just created to do exactly that:

In [ ]:
local price_mean = r(mean)
g price_demean = price - `price_mean'

Note that there is no output when we run this command. 

If we try to run this command a second time, we will get an error because Stata doesn't want us to accidentally overwrite an existing variable. In order to correct this problem, we need to use the command `replace` instead of the command `generate`. Try it yourself above!

Let's take a look at the mean of our new variable using `summarize` again. 

In [ ]:
su price_demean

We can see that the mean is roughly zero just we as expected.

Locals are automatically generated whenever we use loops (as discussed in [Module 3](econometrics/econ490-stata/3_Stata_Essentials.ipynb)). 

Consider another common application here involving a categorical variable that can take on 5 possible values. 

In [ ]:
su rep78

Note that if we run the command above that we used to display the mean of price, we will now get a different value. Try it yourself!

There are times when we might want to save all the possible categorical values in a local. When we use the `levelsof` command as is done below, we can create a new local with a name that we choose. Here, that name is _levels_rep78_.

In [ ]:
levelsof rep78, local(levels_rep)

We can do different things with this new list of values. For instance, we can now summarize a variable based on every distinct value of _rep78_, by creating a loop using `foreach` and looping through values of the newly created local. 

In [ ]:
foreach x in `levels_rep' {
su price if rep78 == `x'
}

## 4.3 Globals

Globals are used to store lists of variable names, paths and/or directories that we need for our research project. 

Consider the following example where we create a global called *covariates* that is simply a list of three variable names:

In [ ]:
global covariates "rep78 foreign"

We can now use this global anywhere we want to invoke the three variables specified. When we want to indicate that we are using a global, we refer to this type of macro with the dollar sign symbol `$`.

Here we `summarize` these three variables. 

In [ ]:
su ${covariates}

In the empty cell below, `describe` these three variables using the macro we have just created. 

## 4.4 Wrap Up

In this module we learned how Stata has its own set of variables that have some very useful applications. We will see these macros throughout the following modules. You will also use them in your own research project.  

To demonstrate how useful macros can be, we can use our _covariates_ global to run a very simple regression in which _price_ is the dependent variable and the explanatory variables are _rep78_ and _foreign_. That command using our macro would be:

In [ ]:
regress price ${covariates}

If we only wanted to include observations where price is above average, then using the local we created earlier in this module the regression would be:

In [ ]:
regress price ${covariates} if price > `price_mean'

You can see for yourself that Stata ran the regression on only a subset of the data.

In the next module, we will work on importing data sets in various formats.